## Created By:- Sameer Goel
## Created On:- 11-04-2021
## Language Used:- Python
## IDE Used:- Jupyter Notebook
## Purpose:- To predict the price of the used car on the basis of some features
## Algorithm Used:- Linear Regression

In [1]:
# Import the important libraries
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score

In [2]:
# Read the data
df = pd.read_csv('car.csv')

In [3]:
# Check the first five rows of the data
df.head()

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing XO eRLX Euro III,Hyundai,2007,"80,000","45,000 kms",Petrol
1,Mahindra Jeep CL550 MDI,Mahindra,2006,"4,25,000",40 kms,Diesel
2,Maruti Suzuki Alto 800 Vxi,Maruti,2018,Ask For Price,"22,000 kms",Petrol
3,Hyundai Grand i10 Magna 1.2 Kappa VTVT,Hyundai,2014,"3,25,000","28,000 kms",Petrol
4,Ford EcoSport Titanium 1.5L TDCi,Ford,2014,"5,75,000","36,000 kms",Diesel


In [4]:
# Check the last five rows of the data
df.tail()

,name,company,year,Price,kms_driven,fuel_type
887,Ta,Tara,zest,"3,10,000",NaN,NaN
888,Tata Zest XM Diesel,Tata,2018,"2,60,000","27,000 kms",Diesel
889,Mahindra Quanto C8,Mahindra,2013,"3,90,000","40,000 kms",Diesel
890,Honda Amaze 1.2 E i VTEC,Honda,2014,"1,80,000",Petrol,NaN
891,Chevrolet Sail 1.2 LT ABS,Chevrolet,2014,"1,60,000",Petrol,NaN


In [5]:
# Check the shape of the data
df.shape

(892, 6)

In [6]:
# Check the information of the data before performing the cleaning operations on the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892 entries, 0 to 891
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        892 non-null    object
 1   company     892 non-null    object
 2   year        892 non-null    object
 3   Price       892 non-null    object
 4   kms_driven  840 non-null    object
 5   fuel_type   837 non-null    object
dtypes: object(6)
memory usage: 41.9+ KB


In [7]:
# Handle the missing values form the data and clean our dataset

# Year
df = df[df['year'].str.isnumeric()]
df['year'] = df['year'].astype(int)

# Price
df = df[df['Price'] != 'Ask For Price']
df['Price'] = df['Price'].str.replace(',','').astype(int)

# Kilometer Driven
df['kms_driven'] = df['kms_driven'].str.split().str.get(0).str.replace(',','')
df = df[df['kms_driven'].str.isnumeric()]
df['kms_driven'] = df['kms_driven'].astype(int)

# fuel type 
df = df[~df['fuel_type'].isna()]

# Taking only first two values from name 
# For example converting Hyundai Santro Xing XO eRLX Euro III to Hyuandai Santro Xing
df['name'] = df['name'].str.split().str.slice(0, 3).str.join(' ')

# Set the index value because after cleaning the data the index is chaged like some values of index are missing
df = df.reset_index(drop = True)

In [8]:
# Check the information of the data after performing the cleaning operations on the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 816 entries, 0 to 815
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        816 non-null    object
 1   company     816 non-null    object
 2   year        816 non-null    int32 
 3   Price       816 non-null    int32 
 4   kms_driven  816 non-null    int32 
 5   fuel_type   816 non-null    object
dtypes: int32(3), object(3)
memory usage: 28.8+ KB


In [9]:
# Check the description of the data
df.describe()

,year,Price,kms_driven
count,816.000000,8.160000e+02,816.000000
mean,2012.444853,4.117176e+05,46275.531863
std,4.002992,4.751844e+05,34297.428044
min,1995.000000,3.000000e+04,0.000000
25%,2010.000000,1.750000e+05,27000.000000
50%,2013.000000,2.999990e+05,41000.000000
75%,2015.000000,4.912500e+05,56818.500000
max,2019.000000,8.500003e+06,400000.000000


In [10]:
# Took only those cars whose price is less than 60 Lakh
df = df[df['Price'] < 6000000]

In [11]:
# Save the cleaned data into a new file called cleaned_data.csv
df.to_csv('cleaned_data.csv')

In [12]:
# Check all the companies of cars avaliable 
companies = df['company'].unique()
print(f'We have {len(companies)} car companies avaliable and the name of the compaines are:-\n{companies}')

We have 25 car companies avaliable and the name of the compaines are:-
['Hyundai' 'Mahindra' 'Ford' 'Maruti' 'Skoda' 'Audi' 'Toyota' 'Renault'
 'Honda' 'Datsun' 'Mitsubishi' 'Tata' 'Volkswagen' 'Chevrolet' 'Mini'
 'BMW' 'Nissan' 'Hindustan' 'Fiat' 'Force' 'Mercedes' 'Land' 'Jaguar'
 'Jeep' 'Volvo']


In [13]:
# Seprate the independent and dependent i.e Price and other factors(name, company, year, kms_drive, fuel_type)
x = df[['name', 'company', 'year', 'kms_driven', 'fuel_type']]
y = df['Price']
print(f'Independent Variables:-\n{x}')
print(f'\n\nDependent Variable:-\n{y}')

Independent Variables:-
                       name   company  year  kms_driven fuel_type
0       Hyundai Santro Xing   Hyundai  2007       45000    Petrol
1       Mahindra Jeep CL550  Mahindra  2006          40    Diesel
2         Hyundai Grand i10   Hyundai  2014       28000    Petrol
3    Ford EcoSport Titanium      Ford  2014       36000    Diesel
4                 Ford Figo      Ford  2012       41000    Diesel
..                      ...       ...   ...         ...       ...
811      Maruti Suzuki Ritz    Maruti  2011       50000    Petrol
812          Tata Indica V2      Tata  2009       30000    Diesel
813    Toyota Corolla Altis    Toyota  2009      132000    Petrol
814            Tata Zest XM      Tata  2018       27000    Diesel
815      Mahindra Quanto C8  Mahindra  2013       40000    Diesel

[815 rows x 5 columns]


Dependent Variable:-
0       80000
1      425000
2      325000
3      575000
4      175000
        ...  
811    270000
812    110000
813    300000
814    2600

In [14]:
# Spliting our data into train and test part
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=655)
print(f'X Train:-\n{x_train}')
print(f'\n\nY Train:-\n{y_train}')
print(f'\n\nY Test:-\n{x_test}')
print(f'\n\nY Test:-\n{y_test}')

X Train:-
                    name   company  year  kms_driven fuel_type
420      Honda Amaze 1.5     Honda  2016      160000    Diesel
722      Ford Fusion 1.4      Ford  2007       85455    Diesel
150   Maruti Suzuki Alto    Maruti  2018        3000    Petrol
347      Mahindra XUV500  Mahindra  2015       15000    Diesel
100  Maruti Suzuki Swift    Maruti  2013       13349    Petrol
..                   ...       ...   ...         ...       ...
121   Maruti Suzuki Alto    Maruti  2017        5600    Petrol
410  Toyota Fortuner 3.0    Toyota  2010      131000    Diesel
185    Maruti Suzuki Zen    Maruti  2000       60000    Petrol
393    Maruti Suzuki Zen    Maruti  2001       40000    Petrol
49   Maruti Suzuki Swift    Maruti  2014       35522    Diesel

[652 rows x 5 columns]


Y Train:-
420     240000
722     125000
150     310000
347    1000000
100     372000
        ...   
121     350000
410     940000
185      55000
393      40000
49      385000
Name: Price, Length: 652, dtype: 

In [15]:
# Seprate categorical features of our data
ohe = OneHotEncoder()
ohe.fit(x[['name', 'company', 'fuel_type']])
print(f'Categorical Features:-\n{ohe.categories_}')

Categorical Features:-
[array(['Audi A3 Cabriolet', 'Audi A4 1.8', 'Audi A4 2.0', 'Audi A6 2.0',
       'Audi A8', 'Audi Q3 2.0', 'Audi Q5 2.0', 'Audi Q7', 'BMW 3 Series',
       'BMW 5 Series', 'BMW 7 Series', 'BMW X1', 'BMW X1 sDrive20d',
       'BMW X1 xDrive20d', 'Chevrolet Beat', 'Chevrolet Beat Diesel',
       'Chevrolet Beat LS', 'Chevrolet Beat LT', 'Chevrolet Beat PS',
       'Chevrolet Cruze LTZ', 'Chevrolet Enjoy', 'Chevrolet Enjoy 1.4',
       'Chevrolet Sail 1.2', 'Chevrolet Sail UVA', 'Chevrolet Spark',
       'Chevrolet Spark 1.0', 'Chevrolet Spark LS', 'Chevrolet Spark LT',
       'Chevrolet Tavera LS', 'Chevrolet Tavera Neo', 'Datsun GO T',
       'Datsun Go Plus', 'Datsun Redi GO', 'Fiat Linea Emotion',
       'Fiat Petra ELX', 'Fiat Punto Emotion', 'Force Motors Force',
       'Force Motors One', 'Ford EcoSport', 'Ford EcoSport Ambiente',
       'Ford EcoSport Titanium', 'Ford EcoSport Trend',
       'Ford Endeavor 4x4', 'Ford Fiesta', 'Ford Fiesta SXi', 'Ford Figo',

In [16]:
# Concatenate our categorical data
column_transformations = make_column_transformer((OneHotEncoder(categories=ohe.categories_), ['name', 'company', 'fuel_type']), remainder='passthrough')

In [17]:
# Train our model
lr = LinearRegression()
pipelining = make_pipeline(column_transformations, lr)
pipelining.fit(x_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['Audi A3 Cabriolet', 'Audi A4 1.8', 'Audi A4 2.0', 'Audi A6 2.0',
       'Audi A8', 'Audi Q3 2.0', 'Audi Q5 2.0', 'Audi Q7', 'BMW 3 Series',
       'BMW 5 Series', 'BMW 7 Series', 'BMW X1', 'BMW X1 sDrive20d',
       'BMW X1 xDrive20d', 'Chevrolet Beat', 'Chevrolet Beat...
                                                                            array(['Audi', 'BMW', 'Chevrolet', 'Datsun', 'Fiat', 'Force', 'Ford',
       'Hindustan', 'Honda', 'Hyundai', 'Jaguar', 'Jeep', 'Land',
       'Mahindra', 'Maruti', 'Mercedes', 'Mini', 'Mitsubishi', 'Nissan',
       'Renault', 'Skoda', 'Tata', 'Toyota', 'Volkswagen', 'Volvo'],
      dtype=object),
                                                                            array(['Diesel', 'LPG', 'Pe

In [18]:
# Making Predictions
predictions = pipelining.predict(x_test)
print(f'Predictions:-\n{predictions}')

Predictions:-
[ 4.93065825e+05  2.00198187e+05  1.89050293e+06  2.26062725e+05
  3.54959522e+05  1.35188799e+05  6.25158693e+05  6.67655859e+05
  5.23151165e+05  3.96454155e+05  1.26456852e+06  7.81885509e+04
  1.29564617e+05  2.31567851e+05  1.87834591e+05  2.81035031e+05
  5.78395185e+05  3.03485052e+05  4.27523099e+05  5.66280423e+05
  4.38010277e+05  3.05224715e+05  2.34096405e+05  1.50463918e+05
  4.37916224e+05  4.53522486e+05  3.00531154e+05  5.12490507e+05
  2.80010962e+05  2.91744495e+05  2.29871523e+05  1.71669713e+05
  2.90573848e+05  3.65808951e+05  1.54166908e+05  5.38093202e+05
 -1.51957031e+02  2.63827875e+05  3.37031316e+05  1.58787716e+05
  1.89062454e+06  6.80336249e+04  3.74864528e+05  2.57721709e+05
  1.71311106e+05  6.70876902e+05  2.43537452e+05  3.78994051e+05
  1.58315438e+05 -5.48135554e+03  1.07523164e+06  1.91806989e+05
  1.52609336e+06  5.65821275e+05  1.97710558e+05  1.71465324e+05
  3.36370264e+05  3.97532203e+04  8.51363724e+05  2.59186787e+05
  5.1938414

In [19]:
# Check accuracy of the model
accuracy = r2_score(y_test, predictions)
print(f'Accuracy of the model:- {round(accuracy * 100, 2)}%')

Accuracy of the model:- 74.07%


In [20]:
# Save the model
pickle.dump(pipelining, open('model.pkl', 'wb'))